In [7]:
%pip install faiss-cpu faiss-gpu-cu12 sentence_transformers transformers accelerate sentencepiece numpy pytorch

  Using cached faiss_cpu-1.12.0-cp311-cp311-win_amd64.whl.metadata (5.2 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement faiss-gpu-cu12 (from versions: none)
ERROR: No matching distribution found for faiss-gpu-cu12

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import faiss
import json
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [3]:
INDEX_FILE = "faiss_index.index"
META_FILE = "meta.jsonl"
MODEL_NAME = "all-MiniLM-L6-v2"

def load_metadata():
    metadata = []
    with open(META_FILE, "r") as f:
        for line in f:
            metadata.append(json.loads(line))
    return metadata

def search(query, top_k=5):
    model = SentenceTransformer(MODEL_NAME)
    query_vector = model.encode(query).astype("float32").reshape(1, -1)
    query_vector /= np.linalg.norm(query_vector, axis=1, keepdims=True)  # Normalize for cosine sim

    index = faiss.read_index(INDEX_FILE)
    metadata = load_metadata()

    D, I = index.search(query_vector, top_k)
    results = []

    for idx in I[0]:
        item = metadata[idx]
        results.append({
            "doc_id": item["doc_id"],
            "chunk_index": item["chunk_index"],
            "text": item["text"]
        })

    return results

# if __name__ == "__main__":
#     query = input("Enter your question: ")
#     results = search(query)

#     print("\nTop matching chunks:\n")
#     for i, r in enumerate(results, 1):
#         print(f"[{i}] {r['text']}\n")
#     print("\nSearch completed.")

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load the LLM for generation
llm_model_id = "microsoft/phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(llm_model_id)
model = AutoModelForCausalLM.from_pretrained(
    llm_model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

def generate_answer(context_chunks, question):
    context = "\n\n".join([chunk['text'] for chunk in context_chunks])
    prompt = f"""You are a helpful assistant. Answer the following question using only the given context. Be brief and factual.\n\nContext:\n{context}\n\nQuestion: {question}\nAnswer:"""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=200, do_sample=True, temperature=0.2,top_p=0.9)

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = decoded.split("Answer:")[-1].strip()
    return answer

c:\Users\adity\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\adity\.cache\huggingface\hub\models--microsoft--phi-3-mini-4k-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. 

In [5]:
if __name__ == "__main__":
    query = input("Enter your question: ")
    results = search(query)

    print("\nTop matching chunks:\n")
    for i, r in enumerate(results, 1):
        print(f"[{i}] {r['text']}\n")

    print("Generating answer...\n")
    answer = generate_answer(results, query)
    print(answer)


Top matching chunks:

[1] Kaladin is the focus character of the novel. At the beginning of his story, he is a squadleader in Amaram's army. His storyline then cuts to his time as a slave. He is sold to  Sadeas's warcamp, where he is placed in Bridge Four, and made to run headlong into enemy fire with the rest of his bridgecrew to give the army a way to cross the chasms as well as drawing enemy arrows away from troops. After a period of despair, Kaladin works to improve the life of his bridgecrew. He trains them so that the bridge runs won't tire them out so much, saves the lives of injured bridgemen, and slowly gets them to start caring again, and to see themselves as soldiers. Kaladin also plans to find a way to escape from Sadeas's army.
Kaladin faces the internal struggles of his inability to save those around him and his distrust of any lighteyes.
As the focus character of The Way of Kings, Kaladin's past is viewed through the use of flashbacks. The flashbacks show his childhood a